This workbook loads and converts a SCANCO .ISQ or .AIM file in the directory to several 
.nrrd formatted images with the following units: density (g/cc), Hounsfield Units, and a
binary thresholded image mask 

Created by: Jason Cox, 04/07/2023

Citations: McCormick, Matthew, et al. "ITK: Enabling Reproducible 
Research and Open Science.” Frontiers in Neuroinformatics, vol. 8, 
Frontiers Media SA, 2014, doi:10.3389/fninf.2014.00013.

In [ ]:
# Install non-default packages
import sys
!{sys.executable} -m pip install itk pooch

In [4]:
# Import required packages
from pathlib import Path
import re
import itk
import pooch
import glob

In [2]:
# ISQ and AIM metadata returned by dict (cell 3) contain the same fields. Therefore, this program could be modified to loop over all files
# in the directory, if that is the desired implementation. The variable all_filenames is unused in the current version, but can be used in a
# modified version that implements a loop.

all_AIM_filenames = glob.glob("*.AIM")
all_ISQ_filenames = glob.glob("*.ISQ")
all_filenames = all_AIM_filenames + all_ISQ_filenames


In [15]:
# This cell originally operated on the image C0005757_SCAP.AIM, though for the purposes of employing pooch, a sample file stored online at the provided URL is now used.
# This section will need to be modified to handle general implementation.

# filename = 'C0005757_SCAP.AIM'
filename = 'C0004255.ISQ'
file_url = 'https://data.kitware.com/api/v1/file/591e56178d777f16d01e0d20/download'
file_sha256 = 'c2a3750c75826cb077d92093d43976cc0350198b55edecd681265eebabfb438b'
file_path = pooch.retrieve(file_url, file_sha256, fname=filename, progressbar=False)
image = itk.imread(file_path)
metadata = dict(image)
HU_pixels = itk.array_view_from_image(image)
# del image
# HU_pixels = np.array(HU_pixels)                     # Array arithmetic below seems to work without this line, but certain operations were not tested as they require too much memory

In [ ]:
# As discussed below, some required constants are missing from metadata. Compare the output of this cell with the file header at:
# https://docs.google.com/document/d/1L2GCnHa4jmK87p0pGI9MNN10MWRMwrk2WqPbPeklP_I/edit
# This section does, however, collect some metadata and write it to a .txt file.

print(metadata)
metadata_str = str(metadata)

metadata_outname = Path(filename).stem + "_metadata.txt"
with open(metadata_outname, "w") as file:
    file.write(metadata_str)

The next cell provides a template file header from C0005757_SCAP.AIM, but this needs to be read from the .AIM or .ISQ.

Then, the following cell identifies numeric constants in the header necessary for converting the image in HU (units given by itk.imread)
to density, or for thresholding/masking the image.

The only constants not readable from the header which must be provided by the user are the per-milli values for low and high thresholds. In basic terms, the low/high threshold pixel values are computed as the respective low/high per-milli value multiplied by the highest pixel value in the image (a quanitity that is given by the header).

In [17]:
header = '{Global Density: intercept=-1.96600006e+02, Global Calib. default unit type=2 (Density), Global Position Slice 1 [um]=47050, Global HU: mu water=0.51400, Global Reconstruction-Alg.=3, Global Parameter units=[1/cm], Global Average data value=322.51215, Global Minimum data value=-2891.00000, Global Created by=ISQ_TO_AIM (IPL), Global Patient Name=JC-NBPI-Rat-May17, Global Energy [V]=70000, Global Scanner ID=2135, Global Standard deviation=0.32443, Global Time=29-JUN-2022 12:39:11.75, Global Orig-ISQ-Dim-p=1792       1792       1781, Global Original Creation-Date=20-AUG-2018 23:18:59.99, Global Parameter name=Linear Attenuation, Global Orig-ISQ-Dim-um=17918      17918      17808, Global Scanner type=10, Global Scan Distance [um]=20480, Global Index Patient=136, Global Index Measurement=7866, Global Minimum value=-0.70581, Global Maximum data value=32767.00000, Global Intensity [uA]=114, Global Maximum value=7.99976, Global Density: slope=3.55519989e+02, Global Mu_Scaling=4096, Global Reference line [um]=47050, Global No. samples=2048, Global Default-Eval=15, Global Site=4, Global Average value=0.07874, Global Calibration Data=Unknown. Backconverted from AIM., Global Standard data deviation=1328.87610, Global No. projections per 180=1000, Global Original file=DK0:[MICROCT.DATA.00000136.00007866]B0005757.ISQ, Global Scaled by factor=4096, Global Angle-Offset [mdeg]=0, Global Integration time [us]=800000, Global Density: unit=mg HA/ccm}'

In [18]:
# Gather numeric constants needed for unit conversion. Currently, the file metadata returned with dict does not contain all of the
# necessary constants. For an example of a full file header with highlighted variables of interest, see the google doc 
# "SCANCO micro-CT Units Conversion" at: https://docs.google.com/document/d/1L2GCnHa4jmK87p0pGI9MNN10MWRMwrk2WqPbPeklP_I/edit

# Thresholds must be entered by the user, they are not contained in the file header
mu_threshold_low = 0.250
mu_threshold_high = 1.0

scanco_mu = float(re.search('Global Scaled by factor=(.*?),', header).group(1))
# scanco_mu = metadata['MuScaling']
max_val = float(re.search('Global Maximum data value=(.*?),', header).group(1))
# max_val = metadata['**********']
density_slope = float(re.search('slope=(.*?),', header).group(1))
# density_slope = metadata['**********']
density_intercept = float(re.search('intercept=(.*?),', header).group(1))
# density_intercept = metadata['**********']
water_mu = float(re.search('water=(.*?),', header).group(1))
# water_mu = metadata['MuWater']

# del header                  # when the code base is modified such that metadata gives all relevant constants, this line and others that reference "header" can be deleted
# del metadata

mu_low = mu_threshold_low * max_val
mu_pixel_low = mu_low/scanco_mu
mu_high = mu_threshold_high * max_val
mu_pixel_high = mu_high/scanco_mu
HU_pixel_low = (-1000) + mu_pixel_low * (1000/water_mu)
HU_pixel_high = (-1000) + mu_pixel_high * (1000/water_mu)
HU_values = [HU_pixel_low, HU_pixel_high]

In [25]:
# Create a binarized version of the input image by applying low and high thresholds. Threshold values are accepted as int type variables, so some rounding is necessary.
lower_threshold = int(round(HU_values[0]))
upper_threshold = int(round(HU_values[1]))
inside_value = 1
outside_value = 0

BW_image = itk.binary_threshold_image_filter(image,lower_threshold=lower_threshold,upper_threshold=upper_threshold,inside_value=inside_value,outside_value=outside_value)

BW_outname = Path(filename).stem + "_BWmask.nrrd"

itk.imwrite(BW_image, BW_outname)

# The following can be enabled to free up memory
# del image
# del BW_image

In [26]:
# Create unit-converted outputs. MUpixels is not generally desired as an output due to representing non-standard units. However, creation of MUpixels
# can be skipped, as RHOpixels can be created by combining the functions needed to convert HU to MU and MU to RHO.

# MUpixels = (HUpixels+1000)/(1000/water_mu)
# RHOpixels = (MUpixels*densitySlope) + densityIntercept
RHO_pixels = ((((HU_pixels+1000)/(1000/water_mu))*density_slope) + density_intercept)/1000  # Dividing by 1000 at the end changes units from mg/cc to g/cc, which is more commonly published


In [29]:
# Create .nrrd file output with units of density (mg/cc or g/cc)

RHO_out = itk.image_view_from_array(RHO_pixels)
for k, v in metadata.items():
    RHO_out[k] = v
RHO_outname = Path(filename).stem + "_RHO.nrrd"
itk.imwrite(RHO_out, RHO_outname)

In [30]:
# Create .nrrd file output with Hounsfield Units (equivalent to loading the file in 3D Slicer)

HU_out = itk.image_view_from_array(HU_pixels)
for k, v in metadata.items():
    HU_out[k] = v
HU_outname = Path(filename).stem + "_HU.nrrd"
itk.imwrite(HU_out, HU_outname)